# json to DataFrame
In this notebook, we create dataframes from the .json Twitter data.
Tweets are extracted with Twitter API version 1.

For each language we have daily files containing raw data.
For each language we create daily files in .tsv as tables: each row is a tweet, and columns are all the attributes needed.

Some files contain more languages, so we have to divide them by language.

Given the json data:
- we skip the malformatted files
- we transform the entries in columns when possible
- we do the same on the nested entries, such as user, retweeted_status, quoted_status
- we do the same on the double nested entries, such as retweeted_status[user]
- we extract urls and hashtags from entities
- we geolocate the tweets with function find_location
- save everything in .../DATA_clean/...

In [1]:
from glob import glob
import pandas as pd
from pandas.core.common import flatten
import json
import csv
import random
import collections
import gzip
import numpy as np
import os
import pytz
import time
import re
import datetime
import unidecode 
from Geolocate_Text import find_location
#import Geolocate_Text
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [4]:
#files to be read are stored in this folder as /date_lang.txt.gz
#some files contain more languages, so we need to check the language for each tweet
files = sorted(glob("/data/fast/public/collections/multilang/DATA/vaccine/*"))
files[0]

'/data/fast/public/collections/multilang/DATA/vaccine/20190904_bg-pl-pt-fr.txt.gz'

In [29]:
langs = sorted(set([u.split("_")[1][:2] for u in glob("/data/fast/public/collections/multilang/DATA/vaccine/*")]))
langs.remove("sk") #sk and hr are empty
langs.remove("hr")

In [3]:
files = sorted(glob("/data/fast/public/collections/multilang/DATA/vaccine/*"))
#how to read a .json file
i = 0
locs = []
with gzip.open(files[24], "rt") as f:
    for line in f:
        tweet = json.loads(line)
        url = tweet["entities"]["urls"]
        if url != []:
            break
tweet

{'created_at': 'Sun Sep 08 04:06:59 +0000 2019',
 'id': 1170549063605006338,
 'id_str': '1170549063605006338',
 'text': '"Die Masern Impfquote in der Ukraine war 2016 mit 31% die niedrigste von Europa, dennoch gab es keine signifikante… https://t.co/HUAlV7wNUE',
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'truncated': True,
 'in_reply_to_status_id': 1165015167337422851,
 'in_reply_to_status_id_str': '1165015167337422851',
 'in_reply_to_user_id': 1054605345082036226,
 'in_reply_to_user_id_str': '1054605345082036226',
 'in_reply_to_screen_name': 'DorisLa34916916',
 'user': {'id': 1054605345082036226,
  'id_str': '1054605345082036226',
  'name': 'Doris Lang',
  'screen_name': 'DorisLa34916916',
  'location': '80638 München',
  'url': None,
  'description': 'T61 ist gefälscht & Tiere müssen weltweit beim Tierarzt verätzen & ersticken das CSU deckt. Seit 1961 bis heute! Habe Heimat verloren & soll verhaftet werden!',
  'translator_type': 'none',
  

In [5]:
#tweet features
tweet.keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'extended_tweet', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms'])

In [7]:
#tweet nested features
tweet["entities"]

{'hashtags': [],
 'urls': [{'url': 'https://t.co/HUAlV7wNUE',
   'expanded_url': 'https://twitter.com/i/web/status/1170549063605006338',
   'display_url': 'twitter.com/i/web/status/1…',
   'indices': [116, 139]}],
 'user_mentions': [],
 'symbols': []}

In [11]:
twt_fields = ["id_str", "created_at", "lang"]

usr_fields = ["id_str", "name", "screen_name", "followers_count","friends_count", 
              "favourites_count", "listed_count", "statuses_count", "verified",
              "location", "created_at", "lang"]

QT_fields = ["id_str", "created_at", "lang"]

QT_usr_fields = ["id_str", "name", "screen_name", "location"]

RT_fields = ["id_str", "created_at", "lang"]

RT_usr_fields = ["id_str", "name", "screen_name", "location"]

cols = list(flatten(["text", twt_fields, "hashtags", ["user_" + field for field in usr_fields], "real_location", "user_country_code",
                     "QT_text", ["QT_" + field for field in QT_fields], ["QT_user_" + field for field in QT_usr_fields],
                     "RT_text", ["RT_" + field for field in RT_fields], ["RT_user_" + field for field in RT_usr_fields],
                      "RT_real_location", "RT_user_country_code"
                    ]))

cols += ["urls"]

Create an empty list outstr.
Each new feature will be appended to the list, if exists, otherwise append " ".

We can create a pandas Series with index cols and entries outstr.

In [12]:
#return the most completed text of the tweet
#if it is a retweet get the text from "retweeted_status"
#if it has a long text, the full text is inside "extended_text"
#otherwise it is in tweet["text"]
#if empty return " "
def get_extended_text(tweet, outstr):
    if "retweeted_status" in tweet:
        txt = get_extended_text(tweet["retweeted_status"], [])[0]
    elif "extended_tweet" in tweet:
        txt = tweet["extended_tweet"]["full_text"]
    elif "text" in tweet:
        txt = tweet["text"]
    else:
        txt = [" "]
    outstr += [txt]
    return outstr

In [13]:
#used to extract all nested text features, such as tweet["retweeted_status"]["text"]
def get_nested_text(tweet, outstr, nested):
    if nested in tweet:
        outstr = get_extended_text(tweet[nested], outstr)
    else:
        outstr += [" "]
    return outstr

In [14]:
#extract simple features tweet["..."]
def get_fields(tweet, outstr, fields):
    for field in fields:
        if field in tweet:
            outstr += [str(tweet[field]).replace("\n"," ").replace("\t"," ")]
        else:
            outstr += [" "]
    return outstr        

In [15]:
#extract nested features tweet["..."]["..."] as strings
def get_nested_fields(tweet, outstr, nested, fields):
    for field in fields:
        if nested in tweet:
            if tweet[nested] != None:
                if field in tweet[nested]:
                    outstr += [str(tweet[nested][field]).replace("\n"," ").replace("\t"," ")]
                else:
                    outstr += [" "]
            else:
                outstr += [" "]
        else:
            outstr += [" "]
    return outstr        

In [16]:
#extract 2 nested features tweet["..."]["..."]["..."]
def get_2nested_fields(tweet, outstr, nested1, nested2, fields):
    for field in fields:
        if nested1 in tweet:
            if nested2 in tweet[nested1]:
                if field in tweet[nested1][nested2]:
                    outstr += [str(tweet[nested1][nested2][field]).replace("\n"," ").replace("\t"," ")]
                else:
                    outstr += [" "]
            else:
                outstr += [" "]
        else:
            outstr += [" "]
    return outstr        


In [17]:
#hashtags are stored as lists in tweet["entities"]["hashtags"]
#extract hashtags in the form "" if no hashtag, "hashtag1", "hashtag1,hastag2" and so on
def get_hashtags(tweet, outstr):
    h = []
    if "entities" in tweet:
        if "hashtags" in tweet["entities"]:
            hashtags = tweet["entities"]["hashtags"]
            for hashtag in hashtags:
                h += [hashtag["text"]]
    outstr += [",".join(h)]
    return outstr

In [22]:
#extract URL in the form "" if no URL, "url1,url2" and so on
def get_url(tweet, outstr):
    url_list = [u["expanded_url"] for u in tweet["entities"]["urls"]]
    outstr += [",".join(url_list)]
    return outstr


In [31]:
#find_location returns location and country code, given a location string
#input: roma, output: (Roma, IT) 
def get_location(tweet, outstr):
    if "location" in tweet["user"]:
        loc_user = find_location(tweet["user"]["location"])
        outstr += [loc_user[0]]
        outstr += [loc_user[1]]
    else:
        outstr += [" "]
        outstr += [" "]
    return outstr

In [24]:
#locate also the retweeted user
def get_RT_location(tweet, outstr):
    if "retweeted_status" in tweet:
        outstr = get_location(tweet["retweeted_status"], outstr)
    else:
        outstr += [" "]
        outstr += [" "]
    return outstr

In [27]:
def get_line(tweet):
    outstr = []
    outstr = get_extended_text(tweet, outstr)
    outstr = get_fields(tweet, outstr, twt_fields)
    outstr = get_hashtags(tweet, outstr)
    outstr = get_nested_fields(tweet, outstr, "user", usr_fields)
    outstr = get_location(tweet, outstr)
    outstr = get_nested_text(tweet, outstr, "quoted_status")
    outstr = get_nested_fields(tweet, outstr, "quoted_status", QT_fields)
    outstr = get_2nested_fields(tweet, outstr, "quoted_status", "user", QT_usr_fields)
    outstr = get_nested_text(tweet, outstr, "retweeted_status")
    outstr = get_nested_fields(tweet, outstr, "retweeted_status", RT_fields)
    outstr = get_2nested_fields(tweet, outstr, "retweeted_status", "user", RT_usr_fields)
    outstr = get_RT_location(tweet, outstr)
    outstr = get_url(tweet, outstr)
    return outstr

In [28]:
get_line(tweet)

['"Die Masern Impfquote in der Ukraine war 2016 mit 31% die niedrigste von Europa, dennoch gab es keine signifikante Anzahl von Masernfälle. Im Jahr 2017 wurden  90% der Kinder in der Ukraine geimpft. Seitdem hat es eine Explosion von Masernfällen gegeben.." https://t.co/rAXz7sEp6w',
 '1170549063605006338',
 'Sun Sep 08 04:06:59 +0000 2019',
 'de',
 '',
 '1054605345082036226',
 'Doris Lang',
 'DorisLa34916916',
 '20',
 '0',
 '21798',
 '0',
 '23294',
 'False',
 '80638 München',
 'Tue Oct 23 05:28:02 +0000 2018',
 'None',
 'Kreisfreie Stadt München',
 'DE',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 'https://twitter.com/i/web/status/1170549063605006338']

In [30]:
pd.Series(get_line(tweet), index = cols)

text                     "Die Masern Impfquote in der Ukraine war 2016 ...
id_str                                                 1170549063605006338
created_at                                  Sun Sep 08 04:06:59 +0000 2019
lang                                                                    de
hashtags                                                                  
user_id_str                                            1054605345082036226
user_name                                                       Doris Lang
user_screen_name                                           DorisLa34916916
user_followers_count                                                    20
user_friends_count                                                       0
user_favourites_count                                                21798
user_listed_count                                                        0
user_statuses_count                                                  23294
user_verified            

In [ ]:
#input: path
#open file, for each tweet check if:
#- it has good format
#- it is outside of 2020 Jan - 2020 Jun
#create get_line()
#save in tsv
def json_tsv_parser(path, write_folder = "/data/public/jlenti/multilang-vax/DATA_clean/"):
    with gzip.open(path, "rt") as this_file:    
        err = 0
        #use try/except procedure to skip the files without correct lines that are not json 
        try:
            for line in this_file:
                line = line.strip()
                outstr = []
                    
                try:
                    tweet = json.loads(line)
                    #skip the lines that are not tweets (if some errors occurred, we have a number, 403 or other)
                    if type(tweet) == int:
                        continue
                    #if created_at is not present there is some errore
                    if "created_at" not in tweet:
                        continue
                    #we focus on Oct-Dec 2019 and Jul 2020-Mar 2021
                    if (pd.to_datetime(tweet["created_at"]) > datetime.datetime(2020,1,1).replace(tzinfo=pytz.UTC)) \
                    & (pd.to_datetime(tweet["created_at"]) < datetime.datetime(2020,6,30).replace(tzinfo=pytz.UTC)):
                        continue
                    #read the lang to save the tweet in the correct folder
                    lang = tweet["lang"]
                    
                    outstr = get_line(tweet)
                    
                    #used to locate the tweet in correct folder
                    right_date = str(pd.to_datetime(tweet["created_at"]).strftime("%Y%m%d")).strip()
                    
                    #save in the format /.../it/20201001-it.tsv
                    write_path = "/".join([write_folder, lang, right_date + "-" + lang + ".tsv"])
                    
                    #write_path = "/home/jlenti/Sample_Data/{0}-{1}.tsv".format(right_date, lang)
                    
                    with open(write_path, "a+") as out_file:
                        tsv_writer = csv.writer(out_file, delimiter='\t')
                        tsv_writer.writerow(outstr)
                    
                except:
                    #print(path.split("/")[8][:11], "err1")
                    err += 1
            print(path.split("/")[8][:11])
        except:
            print(path.split("/")[8][:11], "err2")
            err += 1